# 0_data_collector.ipynb

Collects Reddit posts based on any query using the pushift.io Reddit API.

- Pushshift API Documentation: https://github.com/pushshift/api
- Time Stamp Converter: https://www.unixtimestamp.com/index.php

# Common functions for data collection

In [33]:
import pandas as pd
import os
import requests
import json
import csv
import time
import datetime
from pprint import pprint

TIME_20200901 = '1598918400' 
TIME_20200301 = '1583020800'
TIME_20190301 = '1551398400'
TIME_20190101 = '1546300800'
TIME_20180101 = '1514764800'
TIME_20170101 = '1483228800'
TIME_20110301 = '1293840000'
TIME_20110101 = '1293840000'
TIME_20090101 = '1230768000' 

DATA_FOLDER = 'data_collection'
os.makedirs(DATA_FOLDER, exist_ok=True)

def get_pushshift_data(search_query, before_timestamp, after_timestamp, sub_reddit_name):
    url = 'https://api.pushshift.io/reddit/search/submission/?q=' + \
        str(search_query) + '&size=1000&after=' + str(after_timestamp) + '&before=' + \
        str(before_timestamp) + '&subreddit=' + str(sub_reddit_name)
    # print(url)
    result = requests.get(url)
    
    # Pushshift has a limit of 200 requests per minute
    try:
        data = json.loads(result.text)
    except:
        print('Exceeding the 200 requests per minute limit.')
        print('Waiting 60 seconds...')
        time.sleep(60)
        result = requests.get(url)
        data = json.loads(result.text)
    return data['data']

def collect_submission_data(submission, flairs):
    try:
        body = submission['selftext']
    except KeyError:
        body = '[removed]'

    if body == '' or body == '[removed]' or body == '[deleted]':
        return
    
    submission_data = list()
    title = submission['title']
    submission_id = submission['id']
    score = submission['score']
    num_comments = submission['num_comments']
    full_link = '=HYPERLINK("' + submission['full_link'] + '")'
    created = datetime.datetime.fromtimestamp(submission['created_utc'])
    try:
        link_flair_text = submission['link_flair_text']
    except KeyError:
        link_flair_text = ''
    
    if len(flairs) == 0 or link_flair_text in flairs:
        submission_data.append((submission_id, title, body, score, num_comments, full_link, created, link_flair_text))
        submissions[submission_id] = submission_data

def update_submissions_file(stats, filename):
    upload_count = 0
    with open(filename, 'w', newline='', encoding='utf-8') as file: 
        file_writer = csv.writer(file, delimiter=',')
        headers = ['ID', 'Title', 'Body', 'Score', 'Comments', 'Link', 'Date', 'flair']
        file_writer.writerow(headers)
        for sub in stats:
            file_writer.writerow(stats[sub][0])
            upload_count += 1
        print(str(upload_count) + " submissions have been uploaded to file: " + filename)

# r/cancer
https://www.reddit.com/r/cancer/ 33.6k, June 2008

In [27]:
sub_reddit = 'cancer'
query = ''
before = TIME_20200901
after = TIME_20190101
flair_set = ['Patient ', '']

csv_filename = DATA_FOLDER + '/' + sub_reddit + '.csv'
excel_filename = DATA_FOLDER + '/' + sub_reddit + '.xlsx'
submissions = {}

data = get_pushshift_data(query, before, after, sub_reddit)

while len(data) > 0:
    for submission in data:
        collect_submission_data(submission, flair_set)
    print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    after = data[-1]['created_utc']
    data = get_pushshift_data(query, before, after, sub_reddit)

update_submissions_file(submissions, csv_filename)

read_file = pd.read_csv(csv_filename)
read_file.to_excel(excel_filename, index=None, header=True)

!open '/Applications/Microsoft Excel.app' $excel_filename

2019-01-04 14:15:36
2019-01-08 12:49:15
2019-01-11 07:53:21
2019-01-15 12:23:48
2019-01-19 15:25:32
2019-01-23 11:50:48
2019-01-27 11:01:34
2019-01-29 22:26:33
2019-02-03 12:39:29
2019-02-06 18:42:10
2019-02-10 10:33:29
2019-02-13 18:30:09
2019-02-17 19:54:53
2019-02-20 18:55:50
2019-02-23 16:18:01
2019-02-27 04:08:27
2019-03-01 19:59:33
2019-03-05 17:23:29
2019-03-08 16:10:19
2019-03-12 04:28:10
2019-03-15 16:14:40
2019-03-19 15:21:46
2019-03-23 04:00:02
2019-03-27 08:41:07
2019-03-31 04:45:11
2019-04-03 17:54:42
2019-04-07 09:56:03
2019-04-11 13:36:51
2019-04-15 18:18:33
2019-04-19 09:37:38
2019-04-23 19:12:07
2019-04-27 14:12:33
2019-05-01 12:18:38
2019-05-05 19:48:45
2019-05-09 05:21:58
2019-05-13 11:45:16
2019-05-16 19:37:07
2019-05-21 05:32:59
2019-05-23 20:21:50
2019-05-28 02:01:05
2019-05-30 22:56:06
2019-06-03 18:37:20
2019-06-06 21:32:54
2019-06-10 21:25:21
2019-06-14 08:45:47
2019-06-17 10:39:12
2019-06-20 13:17:50
2019-06-24 12:45:08
2019-06-27 19:48:37
2019-07-01 13:07:04


# r/COVID19positive
https://www.reddit.com/r/COVID19positive/ 84.5k, Mar 2020

In [28]:
sub_reddit = 'COVID19positive'
query = ''
before = TIME_20200901
after = TIME_20200301
flair_set = ['Tested Positive', 'Tested Positive - Family', 'Tested Positive - Friends', 'Tested Positive - Me',
             'Presumed Positive - From Doctor', 'Presumed Positive - From Test']

csv_filename = DATA_FOLDER + '/' + sub_reddit + '.csv'
excel_filename = DATA_FOLDER + '/' + sub_reddit + '.xlsx'
submissions = {}

data = get_pushshift_data(query, before, after, sub_reddit)

while len(data) > 0:
    for submission in data:
        collect_submission_data(submission, flair_set)
    print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    after = data[-1]['created_utc']
    data = get_pushshift_data(query, before, after, sub_reddit)

update_submissions_file(submissions, csv_filename)

read_file = pd.read_csv(csv_filename)

read_file.to_excel(excel_filename, index=None, header=True)

!open '/Applications/Microsoft Excel.app' $excel_filename

2020-03-18 18:06:59
2020-03-19 03:07:06
2020-03-19 12:18:04
2020-03-20 05:47:52
2020-03-21 11:19:37
2020-03-22 14:47:51
2020-03-23 17:23:13
2020-03-24 16:18:25
2020-03-25 16:27:00
2020-03-26 15:01:42
2020-03-27 15:33:27
2020-03-28 17:55:43
2020-03-29 21:48:28
2020-03-30 23:37:42
2020-04-01 06:46:10
2020-04-02 09:35:15
2020-04-03 09:44:02
2020-04-04 10:53:57
2020-04-05 12:06:13
2020-04-06 15:12:37
2020-04-07 22:24:14
2020-04-09 09:16:07
2020-04-10 17:22:40
2020-04-12 11:01:40
2020-04-13 23:06:44
2020-04-15 09:18:49
2020-04-16 21:36:03
2020-04-18 08:48:25
2020-04-19 17:34:00
2020-04-21 11:00:30
2020-04-22 14:38:48
2020-04-24 10:33:28
2020-04-25 12:05:13
2020-04-26 18:25:15
2020-04-28 08:39:06
2020-04-29 10:54:42
2020-04-30 16:15:38
2020-05-01 18:12:41
2020-05-02 23:44:48
2020-05-04 10:06:29
2020-05-05 13:23:53
2020-05-06 20:33:59
2020-05-08 04:58:16
2020-05-09 08:49:31
2020-05-10 17:06:38
2020-05-11 16:35:05
2020-05-12 21:06:46
2020-05-14 01:45:54
2020-05-15 10:42:53
2020-05-16 14:18:44


# r/diabetes
https://www.reddit.com/r/diabetes/, 60.6k, Apr 2008

In [29]:
sub_reddit = 'diabetes'
query = ''
before = TIME_20200901
after = TIME_20090101
flair_set = ['Type 1', 'Type 1.5/LADA', 'Type 2', 'Type 3']

csv_filename = DATA_FOLDER + '/' + sub_reddit + '.csv'
excel_filename = DATA_FOLDER + '/' + sub_reddit + '.xlsx'
submissions = {}

data = get_pushshift_data(query, before, after, sub_reddit)

while len(data) > 0:
    for submission in data:
        collect_submission_data(submission, flair_set)
    print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    after = data[-1]['created_utc']
    data = get_pushshift_data(query, before, after, sub_reddit)

update_submissions_file(submissions, csv_filename)

read_file = pd.read_csv(csv_filename)
read_file.to_excel(excel_filename, index=None, header=True)

!open '/Applications/Microsoft Excel.app' $excel_filename

2010-08-30 03:39:02
2011-01-28 19:57:29
2011-03-22 07:56:15
2011-05-09 21:08:46
2011-06-23 21:24:39
2011-07-28 15:41:00
2011-09-05 00:22:20
2011-10-09 22:20:58
2011-11-02 15:03:37
2011-11-29 13:06:49
2011-12-22 13:24:21
2012-01-11 23:02:30
2012-01-27 15:10:17
2012-02-09 13:19:33
2012-02-22 04:42:18
2012-03-05 06:57:18
2012-03-21 22:31:12
2012-04-03 15:17:05
2012-04-18 11:12:08
2012-05-02 01:39:05
2012-05-16 01:02:43
2012-05-30 07:32:24
2012-06-13 21:14:16
2012-06-27 06:10:45
2012-07-07 17:55:42
2012-07-17 15:00:40
2012-07-27 15:05:04
2012-08-08 06:26:56
2012-08-18 11:52:27
2012-08-30 15:27:43
2012-09-11 05:01:32
2012-09-22 18:27:49
2012-10-02 16:28:18
2012-10-15 00:59:08
2012-10-25 14:56:00
2012-11-06 21:48:31
2012-11-19 15:16:57
2012-11-30 13:09:10
2012-12-12 14:20:05
2012-12-25 22:10:50
2013-01-07 01:32:54
2013-01-14 06:15:03
2013-01-23 12:30:44
2013-02-01 15:39:20
2013-02-11 00:03:08
2013-02-18 11:11:14
2013-02-26 15:13:08
2013-03-04 19:58:07
2013-03-13 08:44:38
2013-03-23 17:44:46


2018-02-21 20:35:43
2018-02-25 09:07:33
2018-03-01 09:18:57
2018-03-04 18:36:52
2018-03-08 07:06:34
2018-03-12 09:17:57
2018-03-15 08:13:57
2018-03-19 02:13:49
2018-03-22 12:11:40
2018-03-26 21:51:34
2018-03-29 17:43:30
2018-04-02 16:22:16
2018-04-05 19:38:06
2018-04-09 17:20:00
2018-04-12 00:19:09
2018-04-15 11:13:15
2018-04-18 02:45:01
2018-04-21 11:08:51
2018-04-25 15:03:27
2018-04-30 11:41:58
2018-05-03 08:24:25
2018-05-07 06:09:08
2018-05-10 13:38:33
2018-05-14 16:09:29
2018-05-17 14:09:36
2018-05-21 18:06:41
2018-05-24 15:44:26
2018-05-29 05:53:22
2018-05-31 17:23:30
2018-06-04 11:56:14
2018-06-07 19:12:16
2018-06-11 17:21:32
2018-06-14 19:39:46
2018-06-18 17:37:23
2018-06-21 14:14:31
2018-06-24 18:57:44
2018-06-27 17:21:51
2018-06-30 20:15:11
2018-07-03 18:28:26
2018-07-07 09:01:55
2018-07-10 21:37:54
2018-07-13 20:35:06
2018-07-17 09:48:10
2018-07-20 04:12:27
2018-07-23 13:29:03
2018-07-26 11:44:44
2018-07-30 05:13:06
2018-08-02 01:49:42
2018-08-04 06:08:30
2018-08-08 03:58:08


# r/eczema
https://www.reddit.com/r/eczema/ 31.4k, May 2010

In [34]:
sub_reddit = 'eczema'
query = ''
before = TIME_20200901
after = TIME_20190301
flair_set = ['biology | symptoms', 'corticosteroid safety', '']

csv_filename = DATA_FOLDER + '/' + sub_reddit + '.csv'
excel_filename = DATA_FOLDER + '/' + sub_reddit + '.xlsx'
submissions = {}

data = get_pushshift_data(query, before, after, sub_reddit)

while len(data) > 0:
    for submission in data:
        collect_submission_data(submission, flair_set)
    print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    after = data[-1]['created_utc']
    data = get_pushshift_data(query, before, after, sub_reddit)

update_submissions_file(submissions, csv_filename)

read_file = pd.read_csv(csv_filename)

# If Excel is needed, have need to convert from csv manually. It encounters an IllegalCharacterError error.
# read_file.to_excel(excel_filename, index=None, header=True)
# !open '/Applications/Microsoft Excel.app' $excel_filename

2019-03-06 12:27:19
2019-03-12 15:28:13
2019-03-19 12:51:16
2019-03-24 17:04:22
2019-03-30 16:38:12
2019-04-03 05:01:19
2019-04-08 11:11:17
2019-04-12 22:01:16
2019-04-17 14:18:26
2019-04-22 13:04:03
2019-04-26 22:09:24
2019-05-01 15:49:13
2019-05-07 09:12:59
2019-05-13 03:29:37
2019-05-18 12:41:15
2019-05-25 23:08:04
2019-05-30 23:26:36
2019-06-05 18:57:09
2019-06-11 19:50:35
2019-06-18 09:09:24
2019-06-23 21:51:53
2019-07-01 00:15:03
2019-07-07 19:51:47
2019-07-13 01:28:09
2019-07-19 21:51:33
2019-07-25 19:10:02
2019-07-31 11:09:45
2019-08-05 12:47:01
2019-08-11 18:48:38
2019-08-18 07:35:45
2019-08-23 12:40:17
2019-08-27 18:24:32
2019-09-02 15:51:11
2019-09-08 00:01:46
2019-09-16 02:48:45
2019-09-21 11:12:10
2019-09-27 01:41:26
2019-10-03 22:23:13
2019-10-09 05:48:33
2019-10-14 21:04:17
2019-10-20 00:26:12
2019-10-24 04:44:24
2019-10-30 07:36:01
2019-11-04 22:34:33
2019-11-10 05:54:13
2019-11-15 08:18:50
2019-11-20 16:13:47
2019-11-26 06:51:53
2019-12-01 17:06:57
2019-12-06 15:02:57


IllegalCharacterError: 

# r/eyetriage
https://www.reddit.com/r/eyetriage/ 1.6k, Jan 2019

In [35]:
sub_reddit = 'eyetriage'
query = ''
before = TIME_20200901
after = TIME_20190101
flair_set = []

csv_filename = DATA_FOLDER + '/' + sub_reddit + '.csv'
excel_filename = DATA_FOLDER + '/' + sub_reddit + '.xlsx'
submissions = {}

data = get_pushshift_data(query, before, after, sub_reddit)

while len(data) > 0:
    for submission in data:
        collect_submission_data(submission, flair_set)
    print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    after = data[-1]['created_utc']
    data = get_pushshift_data(query, before, after, sub_reddit)

update_submissions_file(submissions, csv_filename)

read_file = pd.read_csv(csv_filename)
read_file.to_excel(excel_filename, index=None, header=True)

!open '/Applications/Microsoft Excel.app' $excel_filename

2019-02-16 17:19:16
2019-03-10 00:39:45
2019-04-08 10:42:03
2019-05-06 12:51:50
2019-06-03 15:24:19
2019-06-26 07:59:28
2019-07-24 14:36:05
2019-08-18 03:59:53
2019-09-17 05:20:36
2019-10-13 20:58:19
2019-11-13 10:26:25
2019-12-08 18:13:55
2020-01-07 14:14:05
2020-01-29 09:13:18
2020-02-16 13:25:57
2020-03-08 14:04:35
2020-03-28 12:24:46
2020-04-12 06:53:04
2020-05-03 19:16:35
2020-05-23 00:48:10
2020-06-08 14:54:35
2020-06-23 08:35:21
2020-07-12 14:04:20
2020-07-28 03:44:08
2020-08-14 11:32:57
2020-08-31 15:01:26
2082 submissions have been uploaded to file: data_collection/eyetriage.csv


# r/GERD
https://www.reddit.com/r/GERD/ 16.8k, Aug 2008

In [36]:
sub_reddit = 'GERD'
query = ''
before = TIME_20200901
after = TIME_20190101
flair_set = []

csv_filename = DATA_FOLDER + '/' + sub_reddit + '.csv'
excel_filename = DATA_FOLDER + '/' + sub_reddit + '.xlsx'
submissions = {}

data = get_pushshift_data(query, before, after, sub_reddit)

while len(data) > 0:
    for submission in data:
        collect_submission_data(submission, flair_set)
    print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    after = data[-1]['created_utc']
    data = get_pushshift_data(query, before, after, sub_reddit)

update_submissions_file(submissions, csv_filename)

read_file = pd.read_csv(csv_filename)
read_file.to_excel(excel_filename, index=None, header=True)

!open '/Applications/Microsoft Excel.app' $excel_filename

2019-01-13 03:17:48
2019-01-25 10:14:26
2019-02-06 14:22:51
2019-02-16 20:44:04
2019-02-27 17:02:15
2019-03-08 12:34:04
2019-03-16 17:25:50
2019-03-26 07:45:04
2019-04-03 08:32:47
2019-04-13 10:23:05
2019-04-23 03:46:55
2019-05-05 00:57:02
2019-05-14 19:09:17
2019-05-22 09:25:31
2019-05-29 13:22:02
2019-06-07 05:15:57
2019-06-17 07:39:06
2019-06-26 08:55:24
2019-07-04 07:36:49
2019-07-12 02:56:22
2019-07-19 08:46:32
2019-07-26 21:50:54
2019-08-02 16:50:14
2019-08-09 21:46:55
2019-08-17 06:10:57
2019-08-22 15:26:56
2019-08-29 13:06:45
2019-09-03 23:37:39
2019-09-11 03:12:40
2019-09-18 02:37:58
2019-09-24 11:27:17
2019-09-29 23:51:30
2019-10-06 06:20:59
2019-10-13 09:25:55
2019-10-19 20:01:21
2019-10-26 09:38:08
2019-11-02 08:00:18
2019-11-07 16:16:31
2019-11-12 14:16:50
2019-11-19 00:03:56
2019-11-25 15:26:35
2019-12-02 12:08:04
2019-12-07 17:50:34
2019-12-12 21:14:59
2019-12-17 15:31:43
2019-12-22 19:50:18
2019-12-28 07:35:30
2020-01-02 11:13:27
2020-01-06 16:36:12
2020-01-10 17:38:07


# r/STD
https://www.reddit.com/r/STD/ 13k, Sept 2010

In [37]:
sub_reddit = 'STD'
query = ''
before = TIME_20200901
after = TIME_20180101
flair_set = ['Text Only', '']

csv_filename = DATA_FOLDER + '/' + sub_reddit + '.csv'
excel_filename = DATA_FOLDER + '/' + sub_reddit + '.xlsx'
submissions = {}

data = get_pushshift_data(query, before, after, sub_reddit)

while len(data) > 0:
    for submission in data:
        collect_submission_data(submission, flair_set)
    print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    after = data[-1]['created_utc']
    data = get_pushshift_data(query, before, after, sub_reddit)

update_submissions_file(submissions, csv_filename)

read_file = pd.read_csv(csv_filename)

# If Excel is needed, have need to convert from csv manually. It encounters an IllegalCharacterError error.
# read_file.to_excel(excel_filename, index=None, header=True)
# !open '/Applications/Microsoft Excel.app' $excel_filename

2018-01-09 01:48:01
2018-01-17 19:37:20
2018-01-27 00:37:21
2018-02-05 21:30:41
2018-02-14 10:29:24
2018-02-21 22:34:17
2018-03-04 08:58:12
2018-03-11 12:54:49
2018-03-18 14:49:20
2018-03-26 12:21:17
2018-04-02 21:36:16
2018-04-10 21:06:04
2018-04-19 00:27:17
2018-04-27 04:26:39
2018-05-05 10:26:04
2018-05-14 06:30:57
2018-05-23 07:25:18
2018-05-31 12:35:11
2018-06-06 16:33:29
2018-06-14 12:45:20
2018-06-22 12:17:27
2018-07-01 02:05:28
2018-07-08 00:27:01
2018-07-13 22:12:47
2018-07-18 23:00:19
2018-07-26 11:43:13
2018-08-02 04:36:56
2018-08-08 11:54:25
2018-08-13 15:32:27
2018-08-19 20:01:39
2018-08-26 19:24:50
2018-09-03 09:21:10
2018-09-11 13:43:01
2018-09-18 01:29:33
2018-09-25 02:34:50
2018-10-01 17:51:34
2018-10-08 12:22:18
2018-10-14 05:16:26
2018-10-21 07:06:52
2018-10-28 09:07:29
2018-11-03 23:41:37
2018-11-09 11:44:03
2018-11-16 22:25:13
2018-11-22 17:57:41
2018-12-01 03:58:58
2018-12-07 19:07:23
2018-12-15 15:32:27
2018-12-25 15:50:54
2019-01-01 21:13:20
2019-01-09 15:17:58


IllegalCharacterError: 